<a href="https://colab.research.google.com/github/sonmiannkim/codingdemo/blob/main/Challenge_Two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-upd

In [42]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Tokens").getOrCreate()

In [43]:
from pyspark.ml.feature import Tokenizer

In [59]:
#create sample Dataframe
dataframe = spark.createDataFrame([(1, "Spark is great"), (1, "We are learning Spark"), (2, "Spark is better than hadoop no doubt")], ["id", "sentence"])
dataframe.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  1|      Spark is great|
|  1|We are learning S...|
|  2|Spark is better t...|
+---+--------------------+



In [60]:
# Tokenize sentences
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenizer

Tokenizer_6cedaab126a0

In [61]:
# Create a function to return the length of a list
def word_list_length(word_list):
    return len(word_list)

In [62]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

#remover.transfrom(sentenceData).show(truncate=False)

In [63]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
dataframe

DataFrame[id: bigint, sentence: string]

In [66]:
tokenized_df = tokenizer.transform(dataframe)
tokenized_df.show(truncate=False)


+---+------------------------------------+--------------------------------------------+
|id |sentence                            |words                                       |
+---+------------------------------------+--------------------------------------------+
|1  |Spark is great                      |[spark, is, great]                          |
|1  |We are learning Spark               |[we, are, learning, spark]                  |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|
+---+------------------------------------+--------------------------------------------+



In [67]:
def word_list_lenth(word_list):
  return len(word_list)

In [70]:
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

In [71]:
# Create a user defined function
count_tokens = udf(word_list_length, IntegerType())

In [72]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")

In [73]:
tokenized_df = tokenizer.transform(dataframe)

In [74]:
tokenized_df.withColumn("tokens", count_tokens(col("words"))).show(truncate=False)

+---+------------------------------------+--------------------------------------------+------+
|id |sentence                            |words                                       |tokens|
+---+------------------------------------+--------------------------------------------+------+
|1  |Spark is great                      |[spark, is, great]                          |3     |
|1  |We are learning Spark               |[we, are, learning, spark]                  |4     |
|2  |Spark is better than hadoop no doubt|[spark, is, better, than, hadoop, no, doubt]|7     |
+---+------------------------------------+--------------------------------------------+------+



In [75]:
 # Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()

In [76]:
sentenceData = spark.createDataFrame([(0, ["Big", "data", "is", "super", "powerfule"]),
                                      (1, ["This", "is", "going", "to", "be", "epic"])], ["id", "raw"])

sentenceData.show(truncate=False)

+---+---------------------------------+
|id |raw                              |
+---+---------------------------------+
|0  |[Big, data, is, super, powerfule]|
|1  |[This, is, going, to, be, epic]  |
+---+---------------------------------+



In [77]:
# Import stop words library
from pyspark.ml.feature import StopWordsRemover

In [80]:
# Run the Remover
remover = StopWordsRemover(inputCol="raw", outputCol="filtered")

In [82]:
remover.transform(sentenceData).show()

+---+--------------------+--------------------+
| id|                 raw|            filtered|
+---+--------------------+--------------------+
|  0|[Big, data, is, s...|[Big, data, super...|
|  1|[This, is, going,...|       [going, epic]|
+---+--------------------+--------------------+

